In [1]:
import os
import cv2
#from scipy import misc

import glob
import math
#import time
import datetime
import random
import numpy as np
import pandas as pd
import pickle

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.grid_search import GridSearchCV

import tensorflow as tf

from tensorflow.contrib import skflow
from tensorflow.contrib.skflow import monitors

import matplotlib.pyplot as plt

    

use_input_cache=False #Change this to false if you want to re-process the images with a new preprocessing settings (like # of pixels, etc)
input_cache_file='/Users/ChrisKleban/distracted-driver/objs-128x96-splitByDriver.pickle'


limit=10


pixels_1=256 # 128
pixels_2=192 # 96


imageSize = pixels_1 * pixels_2
num_features = imageSize

my_steps=2000
my_learning_rate=0.015
my_batch_size=64


driversToPutInCV = []
driversToPutInCV.append('p081')
driversToPutInCV.append('p012')
driversToPutInCV.append('p016')



try_finding_edges=True
try_smoothing=True






'''
Read Driver_imgs_list
'''
def get_driver_data():
    dr = dict()
    path = os.path.join('/Users/ChrisKleban/distracted-driver', 'driver_imgs_list.csv')
    print('Read drivers data')
    f2 = open(path, 'r')
    line = f2.readline()
    while (1):
        line = f2.readline()
        if line == '':
            break
        arr = line.strip().split(',')
        dr[arr[2]] = arr[0]
    f2.close()
    return dr


imagesTodrivers=get_driver_data()
#print imagesTodrivers['img_97080.jpg']


#raise SystemExit(0)



'''
Loading data functions
'''

def load_train_cv(encoder,limit,imagesTodrivers,driversToPutInCV):
    X_train = []
    y_train = []
    X_cv = []
    y_cv = []

    print('Read train images')
    for j in range(10):
        print('Load folder c{}'.format(j))
        path = os.path.join('/Users/ChrisKleban/distracted-driver', 'input', 'train', 'c' + str(j), '*.jpg')
        files = glob.glob(path)
        x=0
        for fl in files:

            #print fl

            ### convert image into grey scale 2dimensional vector
            img = cv2.imread(fl,0)
            ### resize image to smaller number of pixels for faster processing
            img = cv2.resize(img, (pixels_1, pixels_2))

            if try_smoothing==True:
                img = cv2.bilateralFilter(img,6,75,75)

            if try_finding_edges==True:
                img = cv2.Canny(img,50,100)


            ### Flatten pixel vector into 1D vector
            #img = img.transpose(2, 0, 1)

            # Old way:  img = np.reshape(img, (1, num_features))
            img = np.reshape(img, (num_features))



            imagename = os.path.basename(fl)
            # If image belongs to driver that goes in CV data set, put it there
            if (imagesTodrivers[imagename] in driversToPutInCV):
                X_cv.append(img)
                y_cv.append(j)


            # If image belongs to driver that goes in training data set, put it there
            else:

                X_train.append(img)
                y_train.append(j)


            # If we are only training a portion of the training data, quit before the end
            x=x+1
            if x > limit:
                break


    ### Put python arrays into NumPy arrays for some reason. WHY????
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_cv = np.array(X_cv)
    y_cv = np.array(y_cv)

    # make sure labels are integers (I think)
    y_train = encoder.fit_transform(y_train).astype('int32')
    y_cv = encoder.fit_transform(y_cv).astype('int32')

    # randomize the order of samples
    X_train, y_train = shuffle(X_train, y_train)
    X_cv, y_cv = shuffle(X_cv, y_cv)

    # split training data in a train and test set
    #X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1)

    #print('Train shape:', X_train.shape, 'CV shape:', X_test.shape)


    ### not sure what this is!!!!!!
    #X_train = X_train.reshape(X_train.shape[0], 1, PIXELS, PIXELS).astype('float32') / 255.
    #X_test = X_test.reshape(X_test.shape[0], 1, PIXELS, PIXELS).astype('float32') / 255.

    #print('Train shape:', X_train.shape, 'CV shape:', X_test.shape)

    return X_train, y_train, X_cv, y_cv, encoder




'''
load training data and start training
'''

if use_input_cache == True:
    print('START: Loading Inputs from Cache')
    ## Create X/Y variables from reading the pickle cached file that we already processed

    # Getting back the objects:
    with open(input_cache_file) as f:
        train_X, train_y, valid_X, valid_y, encoder, X_test, X_test_id = pickle.load(f)
    print('DONE: Loading Inputs from Cache')
          
else:
    print('Loading Inputs by processing images')
    ## Create X/Y variables from processing the images

    encoder = LabelEncoder()

    # load the training and validation data sets
    train_X, train_y, valid_X, valid_y, encoder = load_train_cv(encoder,limit,imagesTodrivers,driversToPutInCV)

    
    


Read drivers data
Loading Inputs by processing images
Read train images
Load folder c0
Load folder c1
Load folder c2
Load folder c3
Load folder c4
Load folder c5
Load folder c6
Load folder c7
Load folder c8
Load folder c9


In [ ]:
print('Train shape:', train_X.shape, 'CV shape:', valid_X.shape)

print('thanks8')


In [3]:

def show_image(im, title='figure'):
    """
    fig = plt.figure()
    
    #plt.ion() #prevent hanging???
    #plt.plot(range(20),range(20))
    
    fig.suptitle(title, fontsize=20)
    
    ####
    
    
    
    fig = plt.figure()
    if im.shape[2]==1:
        im = im.reshape(im.shape[0],im.shape[1])
        plt.imshow(im,cmap='Greys_r')
    else:
        plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
    fig.suptitle(title, fontsize=20)
    """

    #plt.figure()
    plt.title(title)
    #plt.xlabel("Training examples")
    
    #plt.ylabel("Score")
    #plt.grid()
    plt.imshow(im, cmap='Greys_r')

    
    #plt.legend(loc="best")
    return plt



image_num = 4
#title = "Image: " + str(image_num) + "Classification: " + str(train_y[image_num])


image=train_X[image_num].reshape((pixels_2, pixels_1))



   
    


print image


#a = np.arange(6).reshape((3, 2))
#image = cv2.Canny(image,100,200)

#show_image(image,title)
#image = cv2.Canny(image,100,200)

title = "Image: " + str(image_num) + "Classification: " + str(train_y[image_num])

show_image(image,title)
#image = cv2.bilateralFilter(image,9,125,125)
#show_image(image,title)

#image = cv2.Canny(image,200,400)
#show_image(image,title)
                

plt.show()


[[  0   0   0 ..., 255 255   0]
 [  0   0   0 ...,   0 255 255]
 [  0   0   0 ...,   0   0   0]
 ..., 
 [  0   0 255 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]]
